# Импорт библиотек

In [76]:
import pandas as pd
import copy
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import numpy as np
from sklearn import metrics
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

# Загрузка данных

In [7]:
Object1DF = pd.read_csv('pv_01.csv', sep = ';')

In [19]:
Object1DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6217 entries, 0 to 6216
Data columns (total 59 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   time_idx                          6217 non-null   float64
 1   hour_of_day                       6217 non-null   float64
 2   hour_of_day_cos                   6217 non-null   float64
 3   hour_of_day_sin                   6217 non-null   float64
 4   month_of_year                     6217 non-null   float64
 5   month_of_year_cos                 6217 non-null   float64
 6   month_of_year_sin                 6217 non-null   float64
 7   season_of_year                    6217 non-null   float64
 8   season_of_year_cos                6217 non-null   float64
 9   season_of_year_sin                6217 non-null   float64
 10  sunposition_thetaZ                6217 non-null   float64
 11  sunposition_solarAzimuth          6217 non-null   float64
 12  sunpos

# Подготовка данных

### 0. Убираем из датасета пустой столбец

In [18]:
Object1DF.drop('Unnamed: 51', axis=1, inplace=True)

### 1. Добавляем в датасет лаг на TotalCloudCoverageAt0 длительностью 1 день

Добавляем в датасет 8 новых стобцов, которые хранят данные TotalCloudCoverageAt0 за прошедший день. То есть когда будем предсказывать следующее значение SolarRadiation, то вместе с прогнозируемым погодным параметром TotalCloudCoverageAt0 будем передавать в модель исторические данные по облачности за прошедший день от текущего момента.

In [8]:
Object1DF['3_HourLagTotalCloudCoverAt0'] = copy.deepcopy(Object1DF['TotalCloudCoverAt0'].shift(+1))
Object1DF['6_HourLagTotalCloudCoverAt0'] = copy.deepcopy(Object1DF['TotalCloudCoverAt0'].shift(+2))
Object1DF['9_HourLagTotalCloudCoverAt0'] = copy.deepcopy(Object1DF['TotalCloudCoverAt0'].shift(+3))
Object1DF['12_HourLagTotalCloudCoverAt0'] = copy.deepcopy(Object1DF['TotalCloudCoverAt0'].shift(+4))
Object1DF['15_HourLagTotalCloudCoverAt0'] = copy.deepcopy(Object1DF['TotalCloudCoverAt0'].shift(+5))
Object1DF['18_HourLagTotalCloudCoverAt0'] = copy.deepcopy(Object1DF['TotalCloudCoverAt0'].shift(+6))
Object1DF['21_HourLagTotalCloudCoverAt0'] = copy.deepcopy(Object1DF['TotalCloudCoverAt0'].shift(+7))
Object1DF['24_HourLagTotalCloudCoverAt0'] = copy.deepcopy(Object1DF['TotalCloudCoverAt0'].shift(+8))

Удаляем первые 8 записей, потому что они содежат NaN

In [39]:
Object1DF.drop(Object1DF.index[:7], inplace=True)

In [41]:
Object1DF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3878 entries, 12 to 6214
Data columns (total 59 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   time_idx                          3878 non-null   float64
 1   hour_of_day                       3878 non-null   float64
 2   hour_of_day_cos                   3878 non-null   float64
 3   hour_of_day_sin                   3878 non-null   float64
 4   month_of_year                     3878 non-null   float64
 5   month_of_year_cos                 3878 non-null   float64
 6   month_of_year_sin                 3878 non-null   float64
 7   season_of_year                    3878 non-null   float64
 8   season_of_year_cos                3878 non-null   float64
 9   season_of_year_sin                3878 non-null   float64
 10  sunposition_thetaZ                3878 non-null   float64
 11  sunposition_solarAzimuth          3878 non-null   float64
 12  sunpo

### 2. Убираем из датасета ночные часы

В ночные часы не происходит выработки и если оставить их, то они снижают точность модели

In [42]:
Object1DF = Object1DF[~Object1DF['hour_of_day'].isin([0.0000000000000000, 0.1428571428571428, 1.0000000000000000])]

In [43]:
Object1DF.head(10)

,time_idx,hour_of_day,hour_of_day_cos,hour_of_day_sin,month_of_year,month_of_year_cos,month_of_year_sin,season_of_year,season_of_year_cos,season_of_year_sin,...,UpperWindDirectionSin,power_normed,3_HourLagTotalCloudCoverAt0,6_HourLagTotalCloudCoverAt0,9_HourLagTotalCloudCoverAt0,12_HourLagTotalCloudCoverAt0,15_HourLagTotalCloudCoverAt0,18_HourLagTotalCloudCoverAt0,21_HourLagTotalCloudCoverAt0,24_HourLagTotalCloudCoverAt0
12,13.0,0.571429,0.455788,1.000000,0.909091,0.017332,0.258819,1.0,0.0,0.0,...,0.954638,0.420576,1.000000,1.000000,1.000000,0.320312,0.242188,0.781250,0.421875,0.703125
13,14.0,0.714286,0.256176,0.889960,0.909091,0.017332,0.258819,1.0,0.0,0.0,...,0.945316,0.243827,0.898438,1.000000,1.000000,1.000000,0.320312,0.242188,0.781250,0.421875
14,15.0,0.857143,0.095371,0.632563,0.909091,0.017332,0.258819,1.0,0.0,0.0,...,0.624366,0.001235,0.984375,0.898438,1.000000,1.000000,1.000000,0.320312,0.242188,0.781250
18,19.0,0.285714,0.838278,0.732544,0.909091,0.017332,0.258819,1.0,0.0,0.0,...,0.713072,0.000000,0.421875,0.546875,0.750000,1.000000,0.984375,0.898438,1.000000,1.000000
19,20.0,0.428571,0.661157,0.944463,0.909091,0.017332,0.258819,1.0,0.0,0.0,...,0.818096,0.158230,0.906250,0.421875,0.546875,0.750000,1.000000,0.984375,0.898438,1.000000
20,21.0,0.571429,0.455788,1.000000,0.909091,0.017332,0.258819,1.0,0.0,0.0,...,0.890761,0.299383,0.531250,0.906250,0.421875,0.546875,0.750000,1.000000,0.984375,0.898438
21,22.0,0.714286,0.256176,0.889960,0.909091,0.017332,0.258819,1.0,0.0,0.0,...,0.983930,0.093416,1.000000,0.531250,0.906250,0.421875,0.546875,0.750000,1.000000,0.984375
22,23.0,0.857143,0.095371,0.632563,0.909091,0.017332,0.258819,1.0,0.0,0.0,...,0.969094,0.000823,1.000000,1.000000,0.531250,0.906250,0.421875,0.546875,0.750000,1.000000
26,27.0,0.285714,0.838278,0.732544,0.909091,0.017332,0.258819,1.0,0.0,0.0,...,0.539943,0.000000,0.367188,0.882812,1.000000,1.000000,1.000000,1.000000,0.531250,0.906250
27,28.0,0.428571,0.661157,0.944463,0.909091,0.017332,0.258819,1.0,0.0,0.0,...,0.566136,0.110082,0.710938,0.367188,0.882812,1.000000,1.000000,1.000000,1.000000,0.531250


### 3. Оставляем только нужные фичи

В датасете сейчас 59 фичей, часть из них это просто преобразования других фич. Оставляем только нужные.

1. 'hour_of_day'
2. 'month_of_year'
3. 'season_of_year'
4. 'sunposition_thetaZ'
5. 'sunposition_solarAzimuth'
6. 'LowerWindSpeed'
7. 'LowerWindDirection'
8. 'TotalCloudCoverAt0'
9. 'DewpointTemperatureAt0'
10. 'TemperatureAt0'
11. 'RelativeHumidityAt0'
12. 'SurfacePressureAt0'
13. 'SnowDepthAt0'
14. 'SolarRadiationGlobalAt0'
15. 'SolarRadiationDirectAt0'
16. 'SolarRadiationDiffuseAt0'
17. '3_HourLagTotalCloudCoverAt0'
18. '6_HourLagTotalCloudCoverAt0'
19. '9_HourLagTotalCloudCoverAt0'
20. '12_HourLagTotalCloudCoverAt0'
21. '15_HourLagTotalCloudCoverAt0'
22. '18_HourLagTotalCloudCoverAt0'
23. '21_HourLagTotalCloudCoverAt0'
24. '24_HourLagTotalCloudCoverAt0'

In [45]:
filtredDF = Object1DF[['hour_of_day', 'month_of_year', 'season_of_year',
                       'sunposition_thetaZ', 'sunposition_solarAzimuth',
                       'LowerWindSpeed', 'LowerWindDirection', 'TotalCloudCoverAt0',
                       'DewpointTemperatureAt0', 'TemperatureAt0', 'RelativeHumidityAt0',
                       'SurfacePressureAt0', 'SnowDepthAt0',
                       'SolarRadiationGlobalAt0', 'SolarRadiationDirectAt0', 'SolarRadiationDiffuseAt0', 'power_normed',
                       '3_HourLagTotalCloudCoverAt0', '6_HourLagTotalCloudCoverAt0', '9_HourLagTotalCloudCoverAt0', '12_HourLagTotalCloudCoverAt0',
                       '15_HourLagTotalCloudCoverAt0', '18_HourLagTotalCloudCoverAt0', '21_HourLagTotalCloudCoverAt0', '24_HourLagTotalCloudCoverAt0']].copy()

In [31]:
filtredDF.head(10)

,hour_of_day,month_of_year,season_of_year,sunposition_thetaZ,sunposition_solarAzimuth,LowerWindSpeed,LowerWindDirection,TotalCloudCoverAt0,DewpointTemperatureAt0,TemperatureAt0,...,SolarRadiationDiffuseAt0,power_normed,3_HourLagTotalCloudCoverAt0,6_HourLagTotalCloudCoverAt0,9_HourLagTotalCloudCoverAt0,12_HourLagTotalCloudCoverAt0,15_HourLagTotalCloudCoverAt0,18_HourLagTotalCloudCoverAt0,21_HourLagTotalCloudCoverAt0,24_HourLagTotalCloudCoverAt0
2,0.285714,0.909091,1.0,1.000000,0.251081,0.410575,0.623444,1.000000,0.597057,0.399884,...,0.000497,0.000000,1.000000,0.968750,NaN,NaN,NaN,NaN,NaN,NaN
3,0.428571,0.909091,1.0,0.708461,0.379841,0.581901,0.669254,0.976562,0.598159,0.406052,...,0.117925,0.035185,1.000000,1.000000,0.968750,NaN,NaN,NaN,NaN,NaN
4,0.571429,0.909091,1.0,0.642045,0.555848,0.653922,0.698923,0.703125,0.559800,0.407030,...,0.267378,0.120988,0.976562,1.000000,1.000000,0.968750,NaN,NaN,NaN,NaN
5,0.714286,0.909091,1.0,0.906693,0.715489,0.580611,0.677020,0.421875,0.523140,0.401475,...,0.209285,0.201235,0.703125,0.976562,1.000000,1.000000,0.968750,NaN,NaN,NaN
6,0.857143,0.909091,1.0,1.000000,0.747869,0.605988,0.673972,0.781250,0.498032,0.369976,...,0.009186,0.004321,0.421875,0.703125,0.976562,1.000000,1.000000,0.968750,NaN,NaN
10,0.285714,0.909091,1.0,1.000000,0.253177,0.491531,0.484772,1.000000,0.518971,0.338093,...,0.000497,0.000000,1.000000,0.320312,0.242188,0.781250,0.421875,0.703125,0.976562,1.000000
11,0.428571,0.909091,1.0,0.713414,0.380225,0.641618,0.508986,1.000000,0.570053,0.348251,...,0.018620,0.051235,1.000000,1.000000,0.320312,0.242188,0.781250,0.421875,0.703125,0.976562
12,0.571429,0.909091,1.0,0.647150,0.555611,0.524506,0.550464,0.898438,0.632471,0.416335,...,0.172790,0.420576,1.000000,1.000000,1.000000,0.320312,0.242188,0.781250,0.421875,0.703125
13,0.714286,0.909091,1.0,0.911027,0.714830,0.566129,0.569430,0.984375,0.634050,0.433590,...,0.192155,0.243827,0.898438,1.000000,1.000000,1.000000,0.320312,0.242188,0.781250,0.421875
14,0.857143,0.909091,1.0,1.000000,0.745783,0.577573,0.710694,1.000000,0.641161,0.401941,...,0.007944,0.001235,0.984375,0.898438,1.000000,1.000000,1.000000,0.320312,0.242188,0.781250


### 3. Проверяем данные на выбросы

In [46]:
filtredDF.loc[filtredDF['SolarRadiationGlobalAt0'] > 0.9]

,hour_of_day,month_of_year,season_of_year,sunposition_thetaZ,sunposition_solarAzimuth,LowerWindSpeed,LowerWindDirection,TotalCloudCoverAt0,DewpointTemperatureAt0,TemperatureAt0,...,SolarRadiationDiffuseAt0,power_normed,3_HourLagTotalCloudCoverAt0,6_HourLagTotalCloudCoverAt0,9_HourLagTotalCloudCoverAt0,12_HourLagTotalCloudCoverAt0,15_HourLagTotalCloudCoverAt0,18_HourLagTotalCloudCoverAt0,21_HourLagTotalCloudCoverAt0,24_HourLagTotalCloudCoverAt0
1452,0.571429,0.363636,0.333333,0.132209,0.565458,0.249852,0.034213,0.031250,0.360944,0.465491,...,0.410129,0.791975,0.046875,0.000000,0.000000,0.132812,0.421875,0.875000,0.664062,1.000000
1588,0.571429,0.363636,0.333333,0.061925,0.571769,0.196809,0.303862,0.625000,0.744072,0.737986,...,0.360228,0.732922,0.007812,0.000000,0.000000,0.007812,0.023438,0.273438,0.265625,0.539062
1604,0.571429,0.363636,0.333333,0.055155,0.572161,0.033531,0.561511,0.000000,0.827188,0.794156,...,0.354767,0.703909,0.000000,0.000000,0.101562,0.687500,0.828125,0.976562,1.000000,0.156250
1620,0.571429,0.363636,0.333333,0.048734,0.572471,0.154406,0.445371,0.242188,0.717469,0.674377,...,0.430238,0.601440,0.390625,0.187500,0.453125,0.710938,0.804688,0.664062,1.000000,1.000000
1628,0.571429,0.363636,0.333333,0.045657,0.572595,0.008593,0.507118,0.179688,0.623324,0.653668,...,0.417329,0.707407,0.000000,0.023438,0.000000,0.226562,0.039062,0.429688,0.398438,0.242188
1676,0.571429,0.363636,0.333333,0.029138,0.572897,0.394661,0.819432,0.390625,0.556026,0.603173,...,0.376117,0.731276,0.000000,0.382812,0.007812,0.437500,0.906250,0.382812,0.687500,0.773438
1708,0.571429,0.454545,0.333333,0.020058,0.572680,0.364682,0.341562,0.828125,0.719823,0.688122,...,0.450348,0.722840,0.054688,0.101562,0.007812,0.328125,0.335938,0.562500,0.257812,0.757812
1724,0.571429,0.454545,0.333333,0.016122,0.572450,0.173968,0.566231,0.164062,0.661126,0.654317,...,0.417329,0.786831,0.195312,0.000000,0.000000,0.328125,0.656250,0.695312,0.585938,0.695312
1732,0.571429,0.454545,0.333333,0.014309,0.572305,0.149476,0.451148,0.156250,0.770185,0.792953,...,0.427507,0.733951,0.804688,0.000000,0.000000,0.000000,0.000000,0.078125,0.015625,0.164062
1740,0.571429,0.454545,0.333333,0.012600,0.572141,0.055132,0.730120,0.140625,0.906957,0.853336,...,0.538232,0.683951,0.953125,0.890625,0.148438,0.460938,0.921875,0.992188,0.296875,0.156250


### 4. Разделяю данные на тренировочные и тестовые

In [47]:
trainingDF = filtredDF.copy()

X = trainingDF.drop(['SolarRadiationGlobalAt0', 'SolarRadiationDirectAt0', 'SolarRadiationDiffuseAt0', 'power_normed'], axis=1)
y = trainingDF['SolarRadiationGlobalAt0']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20, shuffle=False)

# Обучение модели

## 1. Обучение модели

Для прогнозирования используем MultiLayerPerceptronRegressor из пакета sklearn

In [48]:
MLP = MLPRegressor(hidden_layer_sizes=(14, 4), activation='relu', solver='lbfgs', max_iter=1000, random_state=20)
MLP.fit(X_train, y_train)

## 2. Делаем прогнозирование на тестовых данных

In [ ]:
prediction = MLP.predict(X_test)

# Смотрим результаты обучения

В третьей колонке можно посмотреть ошибку между действительными и спрогнозированными значениями в процентах

In [53]:
crossChecking = pd.DataFrame({'Actual': y_test , 'Predicted': prediction, 'Error': ((((y_test-prediction)**2)**0.5)/y_test)*100})

In [51]:
crossChecking.head(200)

,Actual,Predicted,Error
4355,0.599063,0.559429,6.616001
4356,0.616568,0.707130,14.688058
4357,0.518229,0.619578,19.556689
4358,0.272048,0.301837,10.949690
4362,0.078270,0.078158,0.142028
...,...,...,...
4667,0.442636,0.416755,5.846957
4668,0.580109,0.699597,20.597375
4669,0.383766,0.562565,46.590558
4670,0.169250,0.307184,81.497413


In [54]:
RMSE = np.sqrt(metrics.mean_squared_error(y_test, prediction))
MAPE = metrics.mean_absolute_percentage_error(y_test, prediction)

meanTarget = np.mean(y_test)

In [55]:
print(meanTarget)
print(RMSE)
print(MAPE)

0.2558118561527758
0.06827947896083336
7768564142393.486


# Визуализация

In [77]:
max_idx = len(prediction)
hours_per_dot = 3
hours_per_day = 24
dots_per_day = int(hours_per_day / hours_per_dot)

def plot_graph(index):
    x_axis = [x * hours_per_dot for x in range(0, dots_per_day)]
    plt.plot(x_axis, prediction[index : index + dots_per_day], label='prediction', marker='o')
    plt.plot(x_axis, y_test[index : index + dots_per_day], label='actual', marker='o')
    plt.ylabel('SolarRadiationDiffuseAt0')
    plt.ylim(0, 1)
    plt.xlabel('Hours')
    plt.legend()
    plt.show()

interact(plot_graph, index=IntSlider(min=0, max=max_idx - dots_per_day , step=1, value=0))

interactive(children=(IntSlider(value=0, description='index', max=1156), Output()), _dom_classes=('widget-inte…

<function __main__.plot_graph(index)>

# По мощности

In [69]:
powerDF = Object1DF[['hour_of_day', 'month_of_year', 'season_of_year',
                 'sunposition_thetaZ', 'sunposition_solarAzimuth',
                 'LowerWindSpeed', 'LowerWindDirection', 'TotalCloudCoverAt0',
                 'DewpointTemperatureAt0', 'TemperatureAt0', 'RelativeHumidityAt0',
                 'SurfacePressureAt0', 'SnowDepthAt0',
                 '3_HourLagTotalCloudCoverAt0', '6_HourLagTotalCloudCoverAt0', '9_HourLagTotalCloudCoverAt0', '12_HourLagTotalCloudCoverAt0',
                 '15_HourLagTotalCloudCoverAt0', '18_HourLagTotalCloudCoverAt0', '21_HourLagTotalCloudCoverAt0', '24_HourLagTotalCloudCoverAt0',
                 'power_normed']].copy()

In [70]:
p_X = powerDF.drop(['power_normed'], axis=1)
p_y = powerDF['power_normed']

p_X_train, p_X_test, p_y_train, p_y_test = train_test_split(p_X, p_y, test_size=0.3, random_state=20, shuffle=False)

In [71]:
powerMLP = MLPRegressor(hidden_layer_sizes=(14, 4), activation='relu', solver='lbfgs', max_iter=1000, random_state=20)
powerMLP.fit(p_X_train, p_y_train)
powerPrediction = powerMLP.predict(p_X_test)

In [72]:
powerRMSE = np.sqrt(metrics.mean_squared_error(p_y_test, powerPrediction))
print(powerRMSE)

powerMeanTarget = np.mean(p_y_test)
print(powerMeanTarget)

0.10151073791127564
0.19644142519762983


In [79]:
max_idx = len(powerPrediction)
hours_per_dot = 3
hours_per_day = 24
dots_per_day = int(hours_per_day / hours_per_dot)

def plot_graph(index):
    x_axis = [x * hours_per_dot for x in range(0, dots_per_day)]
    plt.plot(x_axis, prediction[index : index + dots_per_day], label='powerPrediction', marker='o')
    plt.plot(x_axis, p_y_test[index : index + dots_per_day], label='actual', marker='o')
    plt.ylabel('SolarRadiationDiffuseAt0')
    plt.ylim(0, 1)
    plt.xlabel('Hours')
    plt.legend()
    plt.show()

interact(plot_graph, index=IntSlider(min=0, max=max_idx - dots_per_day , step=1, value=0))

interactive(children=(IntSlider(value=0, description='index', max=1156), Output()), _dom_classes=('widget-inte…

<function __main__.plot_graph(index)>